In [1]:
import os
import sys

# Obtener el directorio actual de la notebook
notebook_dir = os.getcwd()

# Obtener el directorio padre
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Agregar el directorio padre al path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from libs.config.config_variables import (
    MAX_WORKERS,
    TIMEOUT_API_REQUEST,
    CATALOG_CSV_DIR,
)

from modules.catalog_scraper.seismic_scraper import SeismicScraper
from modules.downloader.records_downloader import SeismicDownloader

# 0. Configuración básica de catálogos

In [3]:
# Límites geográficos (Perú y alrededores)
MIN_LAT, MAX_LAT = -20.0, 0.0
MIN_LON, MAX_LON = -85.0, -68.0

# Año de inicio del scraping
START_YEAR = 1400

# 1. Ejecución de scraping de catálogos


## 1.1 Carga de datos históricos

In [4]:
# Registrar los catálogos csv
historical_scraper = SeismicScraper(
    max_workers=MAX_WORKERS, timeout=TIMEOUT_API_REQUEST, csv_base_path=CATALOG_CSV_DIR
)
results = historical_scraper.scrape_events_from_csv(
    catalogs=["gcmt", "isc-gem", "sara"],
    # start_year=START_YEAR,
    # min_lat=MIN_LAT,
    # max_lat=MAX_LAT,
    # min_lon=MIN_LON,
    # max_lon=MAX_LON,
)


2025-11-24 14:21:00,049 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events_from_csv
2025-11-24 14:21:00,051 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 1 archivos CSV para GCMT
2025-11-24 14:21:00,052 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para ISC-GEM
2025-11-24 14:21:00,053 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para SARA
2025-11-24 14:21:00,053 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: 5 archivos con 40 workers
2025-11-24 14:21:00,055 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: GCMT - gcmt_data.csv
2025-11-24 14:21:00,056 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-cat.csv
2025-11-24 14:21:00,075 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-suppl.csv
2025-11-24 14:21:00,076 [INFO] modules.ca

In [5]:
historical_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|     236356      | 64879  |    171477    |    1    |
+-----------------+--------+--------------+---------+


## 1.2 Actualización de datos recientes

In [6]:
recent_data_scraper = SeismicScraper(
    max_workers=MAX_WORKERS, timeout=TIMEOUT_API_REQUEST
)

results = recent_data_scraper.scrape_events(
    start_year=2000,
    min_lat=MIN_LAT,
    max_lat=MAX_LAT,
    min_lon=MIN_LON,
    max_lon=MAX_LON,
    catalogs=["usgs", "isc", "igp"],
)

2025-11-24 14:21:18,519 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events
2025-11-24 14:21:18,520 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: 78 tareas con 40 workers
2025-11-24 14:21:18,523 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2000
2025-11-24 14:21:18,523 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2001
2025-11-24 14:21:18,530 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2002
2025-11-24 14:21:18,532 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2003
2025-11-24 14:21:18,546 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2004
2025-11-24 14:21:18,546 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2005
2025-11-24 14:21:18,547 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2006
2025-11-24 14:21:18,551 [INFO] m

In [7]:
recent_data_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|      16020      |  1692  |    14328     |   15    |
+-----------------+--------+--------------+---------+


## 1.3 Descarga de registros acelerométricos

In [8]:
NUM_EVENTS = 100
PARALLEL_STATIONS = 10
CATALOG_PARSER_CONFIG = {
    "igp": 6000,
    # "usgs": 5,
    # "emsc": 15,
}

In [9]:
downloader = SeismicDownloader()

downloader.process_multiple_catalogs(
    catalogs=CATALOG_PARSER_CONFIG,
    max_workers=MAX_WORKERS,
    parallel_stations=PARALLEL_STATIONS,
)

2025-11-24 14:21:34,792 [INFO] modules.downloader.records_downloader: Starting execution of SeismicDownloader.process_multiple_catalogs
2025-11-24 14:21:34,793 [INFO] modules.downloader.records_downloader: Iniciando procesamiento de 1 catálogos
2025-11-24 14:21:34,794 [INFO] modules.downloader.records_downloader: Procesando catálogo: IGP (6000 eventos)
2025-11-24 14:21:34,795 [INFO] modules.downloader.records_downloader: Starting execution of SeismicDownloader.process_events
2025-11-24 14:21:34,817 [INFO] modules.downloader.records_downloader: Eventos a procesar en IGP: 5297
2025-11-24 14:21:34,818 [INFO] modules.downloader.records_downloader: Starting execution of SeismicDownloader.process_catalog
2025-11-24 14:21:34,819 [INFO] modules.downloader.records_downloader: Iniciando procesamiento: IGP - 5297 eventos
2025-11-24 14:21:34,821 [INFO] libs.database.seismic_records: Inicio de sesión de sincronización (record_id=2, catalog_id=276)
2025-11-24 14:21:34,824 [WARNING] modules.downloade

{'success': True,
 'catalogs_processed': 1,
 'results': {'igp': {'result': {'success': True,
    'catalog': 'igp',
    'sync_id': 2,
    'status': 'completed_with_errors',
    'total_events': 5297,
    'processed_events': 5297,
    'successful_events': 0,
    'failed_events': 5297,
    'total_stations': 0,
    'total_samples': 0,
    'processing_time': 18066.36102628708,
    'errors': ['2025-0002: Procesamiento fallido',
     '2025-0010: Procesamiento fallido',
     '2025-0011: Procesamiento fallido',
     '2025-0008: Procesamiento fallido',
     '2025-0006: Procesamiento fallido',
     '2025-0012: Procesamiento fallido',
     '2025-0005: Procesamiento fallido',
     '2025-0020: Procesamiento fallido',
     '2025-0016: Procesamiento fallido',
     '2025-0019: Procesamiento fallido']},
   'total_events': 5297,
   'processed_events': 5297,
   'successful_events': 0,
   'failed_events': 5297,
   'total_stations': 0,
   'total_samples': 0,
   'processing_time': 18066.36102628708}}}

In [10]:
downloader.metadata

+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+------------------+-------------------+---------------+----------------+---------------+-------------------+
| Catalogo |                                                                                                                                                                                                                           